In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split,window,current_timestamp,col
from pyspark.sql.types import TimestampType

In [ ]:
sc = SparkContext("local[2]", "ContarOalavrasStreaming")

def atualizer(novos_valores, ultimos_valores):
    return sum(novos_valores) + (ultimos_valores or 0)

ssc = StreamingContext(sc,10)

dados = ssc.socketTextStream("localhost", 3456)

palavras = dados.flatMap(lambda linha: linha.split(" "))
pares = palavras.map(lambda palavra: (palavra, 1))
contagem = pares.updateStateByKey(atualizer)

contagem.pprint()

ssc.checkpoint("checkpoint")

ssc.start()

ssc.awaitTermination()

In [ ]:
from pyspark import SparkContext


sc = SparkContext('local[2]', 'Contarstreaming')
sc.setLogLevel('ERROR')

spark = SparkSession.builder.appName("Teste").getOrCreate()

linhas = spark.readStream \
    .format("socket") \
    .option("host","localhost") \
    .option("port",3456)\
    .option("includeTimestamp", "true")\
    .load()


palavras = linhas.select(
            explode(split(linhas.value, " ")).alias("palavra"),
            linhas.timestamp
)

contagem = palavras \
    .groupBy(
        window(col('timestamp'), "2 minutes", "1 minutes"),
        col('palavra')) \
    .count()


consulta = contagem.writeStream\
    .outputMode("append")\
    .format("csv")\
    .option("checkpointLocation", "checkpoint_folder")\
    .option("path", "teste")\
    .start()\
    .awaitTermination()
    



In [ ]:


sc = SparkContext("local[2]", "ContarOalavrasStreaming")

ssc = StreamingContext(sc,10)

dados = ssc.socketTextStream("localhost", 3456)

palavras = dados.flatMap(lambda linha: linha.split(" "))
pares = palavras.map(lambda palavra: (palavra, 1))
contagem = pares.reduceByKey(lambda x,y : x+y)

contagem.pprint()



ssc.start()

ssc.awaitTermination()